In [ ]:
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.5.2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve,auc,classification_report,make_scorer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from link_prediction import *

In [ ]:
#Load the entity embeddings dict
with open("fb15k237_distmult_entity_embeddings.pkl", "rb") as f:
    entity_embeddings = pickle.load(f)
#Load the predicate embeddings dict
with open("fb15k237_distmult_predicate_embeddings.pkl", "rb") as f:
    predicate_embeddings = pickle.load(f)

train_triples = pd.read_csv('fb15k237_train.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])
test_triples = pd.read_csv('fb15k237_test.txt', dtype=str, sep='\t', header=None, names=['head', 'relation', 'tail'])

**Create Negative Samples**

In [ ]:
training_object=Training(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='DistMult')
tail_df = training_object.create_training_data_filtered(n=1, creating_for="tail")
head_df = training_object.create_training_data_filtered(n=1, creating_for="head")
print(len(tail_df))
print(len(head_df))

Processing triples: 100%|██████████| 272115/272115 [01:43<00:00, 2617.11triple/s]

93370
56314


**Prepare Training Data**

In [ ]:
train_triples['label']=1
train_df = pd.concat([head_df,tail_df,train_triples], axis=0)
# Reset the index
train_df.reset_index(drop=True, inplace=True)
# Drop duplicate rows
train_df.drop_duplicates(inplace=True)
# Step 1: Prepare the training data
train_df['embedding'] = train_df.apply(lambda row: training_object.get_embedding(row).cpu().numpy(), axis=1)
X_train = np.vstack(train_df['embedding'].values)  # Stack embeddings into a matrix
y_train = train_df['label'].values  # Labels
# Step 1: Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_df['embedding'].values),  # Embeddings matrix
    train_df['label'].values,                # Labels
    test_size=0.1,                           # 10% of the data for testing
    random_state=42,                         # For reproducibility
    stratify=train_df['label'].values        # Maintain label distribution
)
train_df['label'].value_counts()

,count
label,
1,272115
0,144977


**Set Models**

**MLP**

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter("ignore", ConvergenceWarning)

In [ ]:
param_grid_mlp = {
    'hidden_layer_sizes': [(256, 256)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001],
    'max_iter': [50]
}



results_mlp = []
for hidden_layer_sizes in param_grid_mlp['hidden_layer_sizes']:
    for alpha in param_grid_mlp['alpha']:
        for learning_rate_init in param_grid_mlp['learning_rate_init']:
            for max_iter in param_grid_mlp['max_iter']:
                mlp = MLPClassifier(
                    hidden_layer_sizes=hidden_layer_sizes,
                    alpha=alpha,
                    learning_rate_init=learning_rate_init,
                    max_iter=max_iter,
                    solver='adam',
                    early_stopping=True,
                    batch_size=32,
                    random_state=42
                )
                mlp.fit(X_train, y_train)
                preds = mlp.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_mlp.append({
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'alpha': alpha,
                    'learning_rate_init': learning_rate_init,
                    'max_iter': max_iter,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_mlp = pd.DataFrame(results_mlp)

best_mlp_params = df_results_mlp.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for MLP:", best_mlp_params)

Best parameters for MLP: {'hidden_layer_sizes': (256, 256), 'alpha': 0.0001, 'learning_rate_init': 0.001, 'max_iter': 50, 'f1_score': 0.8313385232689735, 'accuracy': 0.7641812514984416, 'precision_0': 0.71982270841192, 'precision_1': 0.7793030283546583, 'recall_0': 0.5264864119188853, 'recall_1': 0.8908202263707188}


**XGB**

In [ ]:
param_grid_xgb = {
    'n_estimators': [300],
    'max_depth': [15],
    'scale_pos_weight': [0.5, 1],
    'learning_rate': [0.01, 0.1]
}

results_xgb = []
for n_estimators in param_grid_xgb['n_estimators']:
    for max_depth in param_grid_xgb['max_depth']:
        for scale_pos_weight in param_grid_xgb['scale_pos_weight']:
            for learning_rate in param_grid_xgb['learning_rate']:
                xgb = XGBClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    scale_pos_weight=scale_pos_weight,
                    learning_rate=learning_rate,
                    random_state=42,
                    #use_label_encoder=False,
                    #eval_metric='logloss'
                )
                xgb.fit(X_train, y_train)
                preds = xgb.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_xgb.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'scale_pos_weight': scale_pos_weight,
                    'learning_rate': learning_rate,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_xgb = pd.DataFrame(results_xgb)
best_xgb_params = df_results_xgb.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for XGBClassifier:", best_xgb_params)

**LGBM**

In [ ]:
param_grid_lgbm = {
    'n_estimators': [300],
    'max_depth': [15],
    'learning_rate': [0.01, 0.1],
    'is_unbalance': [True, False]
}

results_lgbm = []
for n_estimators in param_grid_lgbm['n_estimators']:
    for max_depth in param_grid_lgbm['max_depth']:
        for learning_rate in param_grid_lgbm['learning_rate']:
            for is_unbalance in param_grid_lgbm['is_unbalance']:
                lgbm = LGBMClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate,
                    is_unbalance=is_unbalance,
                    random_state=42
                )
                lgbm.fit(X_train, y_train)
                preds = lgbm.predict(X_test)

                # Calculate metrics
                score_f1 = f1_score(y_test, preds)
                acc = accuracy_score(y_test, preds)
                precisions = precision_score(y_test, preds, average=None, zero_division=0)
                recalls = recall_score(y_test, preds, average=None, zero_division=0)

                results_lgbm.append({
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'learning_rate': learning_rate,
                    'is_unbalance': is_unbalance,
                    'f1_score': score_f1,
                    'accuracy': acc,
                    'precision_0': precisions[0],
                    'precision_1': precisions[1],
                    'recall_0': recalls[0],
                    'recall_1': recalls[1]
                })

df_results_lgbm = pd.DataFrame(results_lgbm)
best_lgbm_params = df_results_lgbm.sort_values(by='f1_score', ascending=False).iloc[0].to_dict()
print("Best parameters for LGBM:", best_lgbm_params)

**Train Ensemble**

In [ ]:
# Best MLPClassifier
best_mlp = MLPClassifier(
    hidden_layer_sizes=best_mlp_params['hidden_layer_sizes'],
    alpha=best_mlp_params['alpha'],
    learning_rate_init=best_mlp_params['learning_rate_init'],
    max_iter=best_mlp_params['max_iter'],
    solver='adam',
    early_stopping=True,
    batch_size=32,
    random_state=42
)
best_mlp.fit(X_train, y_train)
preds_mlp = best_mlp.predict(X_test)
report_mlp = classification_report(y_test, preds_mlp)

# Best XGBClassifier
best_xgb = XGBClassifier(
    n_estimators=int(best_xgb_params['n_estimators']),
    max_depth=int(best_xgb_params['max_depth']),
    learning_rate=float(best_xgb_params['learning_rate']),
    scale_pos_weight=float(best_xgb_params['scale_pos_weight']),
    random_state=42,
)
best_xgb.fit(X_train, y_train)
preds_xgb = best_xgb.predict(X_test)
report_xgb = classification_report(y_test, preds_xgb)

# Best LGBMClassifier
best_lgbm = LGBMClassifier(
    n_estimators=best_lgbm_params['n_estimators'],
    max_depth=best_lgbm_params['max_depth'],
    learning_rate=best_lgbm_params['learning_rate'],
    is_unbalance=best_lgbm_params['is_unbalance'],
    random_state=42
)
best_lgbm.fit(X_train, y_train)
preds_lgbm = best_lgbm.predict(X_test)
report_lgbm = classification_report(y_test, preds_lgbm)

######################################
# Ensemble the best models using VotingClassifier
######################################
ensemble_model = VotingClassifier(
    estimators=[
        ('mlp', best_mlp),
        ('xgb', best_xgb),
        ('lgbm', best_lgbm)
    ],
    voting='soft',  # Soft voting uses predicted probabilities
    n_jobs=-1
)

start_time = time.time()
ensemble_model.fit(X_train, y_train)
end_time = time.time()

ensemble_time = end_time - start_time
preds_ensemble = ensemble_model.predict(X_test)
ensemble_acc = accuracy_score(y_test, preds_ensemble)
report_ensemble = classification_report(y_test, preds_ensemble)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 244903, number of negative: 130479
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.513456 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 74517
[LightGBM] [Info] Number of data points in the train set: 375382, number of used features: 300
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.652410 -> initscore=0.629650
[LightGBM] [Info] Start training from score 0.629650


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
y_pred_ensemble = ensemble_model.predict(X_test)
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Ensemble Classification Report:\n", classification_report(y_test, y_pred_ensemble))

Ensemble Accuracy: 0.7939582833852793
Ensemble Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.59      0.66     14498
           1       0.80      0.90      0.85     27212

    accuracy                           0.79     41710
   macro avg       0.78      0.75      0.76     41710
weighted avg       0.79      0.79      0.79     41710



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
with open("/content/drive/MyDrive/link_prediction/tuning/fb15k237_distmult_tuning.txt", "w") as f:
    f.write("##############################\n")
    f.write("MLPClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_mlp.to_string())
    f.write("\n\nBest Parameters for MLPClassifier:\n")
    f.write(str(best_mlp_params))
    f.write("\n\nClassification Report for Best MLPClassifier:\n")
    f.write(report_mlp)

    f.write("\n\n\n##############################\n")
    f.write("XGBClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_xgb.to_string())
    f.write("\n\nBest Parameters for XGBClassifier:\n")
    f.write(str(best_xgb_params))
    f.write("\n\nClassification Report for Best XGBClassifier:\n")
    f.write(report_xgb)

    f.write("\n\n\n##############################\n")
    f.write("LGBMClassifier Full Results\n")
    f.write("##############################\n")
    f.write(df_results_lgbm.to_string())
    f.write("\n\nBest Parameters for LGBMClassifier:\n")
    f.write(str(best_lgbm_params))
    f.write("\n\nClassification Report for Best LGBMClassifier:\n")
    f.write(report_lgbm)

    f.write("\n\n\n##############################\n")
    f.write("Ensemble Model Performance\n")
    f.write("##############################\n")
    f.write("Ensemble Accuracy: {:.4f}\n".format(ensemble_acc))
    f.write("Ensemble Classification Report:\n")
    f.write(report_ensemble)

print("Results saved to fb15k237_distmult_tuning.txt")

Results saved to fb15k237_distmult_tuning.txt


**Rerank Hitk**

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
tripleEvaluator=TripleEvaluator(entity_embeddings,predicate_embeddings,train_triples,test_triples, model='DistMult', k=50)

Precomputing top-k predictions: 100%|██████████| 20466/20466 [00:32<00:00, 626.60it/s]


In [ ]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2,3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9,10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10,20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19,20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_mlp, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 13.82% | tail: 24.38% | overall: 20.16%
k=1 | th=2 | head: 14.38% | tail: 27.06% | overall: 22.00%
k=1 | th=3 | head: 13.94% | tail: 27.76% | overall: 22.25%
k=1 | th=4 | head: 13.50% | tail: 27.67% | overall: 22.01%
k=1 | th=5 | head: 13.34% | tail: 27.39% | overall: 21.78%
k=1 | th=6 | head: 13.18% | tail: 27.21% | overall: 21.61%
k=1 | th=7 | head: 12.35% | tail: 26.71% | overall: 20.98%
k=1 | th=8 | head: 11.61% | tail: 26.71% | overall: 20.69%
k=1 | th=9 | head: 11.30% | tail: 26.20% | overall: 20.26%
k=1 | th=10 | head: 11.04% | tail: 25.66% | overall: 19.83%
k=1 | th=20 | head: 8.24% | tail: 23.34% | overall: 17.31%
k=1 | th=30 | head: 7.05% | tail: 22.58% | overall: 16.38%
k=1 | th=40 | head: 6.20% | tail: 22.07% | overall: 15.74%
k=1 | th=50 | head: 5.69% | tail: 21.50% | overall: 15.19%
k=3 | th=3 | head: 26.49% | tail: 39.87% | overall: 34.53%
k=3 | th=4 | head: 26.27% | tail: 40.56% | overall: 34.85%
k=3 | th=5 | head: 25.93% | tail: 40.57% | overall: 34.

In [ ]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_xgb, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 13.82% | tail: 24.38% | overall: 20.16%
k=1 | th=2 | head: 18.81% | tail: 31.62% | overall: 26.51%
k=1 | th=3 | head: 18.22% | tail: 31.53% | overall: 26.22%
k=1 | th=4 | head: 17.65% | tail: 31.03% | overall: 25.69%
k=1 | th=5 | head: 16.95% | tail: 30.48% | overall: 25.08%
k=1 | th=6 | head: 16.43% | tail: 30.30% | overall: 24.77%
k=1 | th=7 | head: 16.05% | tail: 29.89% | overall: 24.36%
k=1 | th=8 | head: 15.35% | tail: 29.65% | overall: 23.94%
k=1 | th=9 | head: 14.92% | tail: 29.05% | overall: 23.41%
k=1 | th=10 | head: 14.71% | tail: 28.79% | overall: 23.17%
k=1 | th=20 | head: 13.02% | tail: 26.54% | overall: 21.14%
k=1 | th=30 | head: 12.23% | tail: 25.67% | overall: 20.31%
k=1 | th=40 | head: 11.77% | tail: 25.33% | overall: 19.91%
k=1 | th=50 | head: 11.61% | tail: 25.09% | overall: 19.71%
k=3 | th=3 | head: 26.49% | tail: 39.87% | overall: 34.53%
k=3 | th=4 | head: 29.25% | tail: 43.00% | overall: 37.52%
k=3 | th=5 | head: 29.22% | tail: 42.99% | overall:

In [ ]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=best_lgbm, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 13.82% | tail: 24.38% | overall: 20.16%
k=1 | th=2 | head: 15.16% | tail: 27.58% | overall: 22.62%
k=1 | th=3 | head: 15.43% | tail: 28.52% | overall: 23.30%
k=1 | th=4 | head: 15.25% | tail: 28.44% | overall: 23.18%
k=1 | th=5 | head: 14.98% | tail: 28.67% | overall: 23.21%
k=1 | th=6 | head: 14.85% | tail: 28.85% | overall: 23.26%
k=1 | th=7 | head: 14.64% | tail: 28.69% | overall: 23.08%
k=1 | th=8 | head: 14.34% | tail: 28.56% | overall: 22.88%
k=1 | th=9 | head: 14.02% | tail: 28.27% | overall: 22.58%
k=1 | th=10 | head: 13.78% | tail: 28.01% | overall: 22.33%
k=1 | th=20 | head: 12.37% | tail: 26.53% | overall: 20.88%
k=1 | th=30 | head: 11.37% | tail: 25.65% | overall: 19.95%
k=1 | th=40 | head: 10.89% | tail: 24.94% | overall: 19.33%
k=1 | th=50 | head: 10.39% | tail: 24.44% | overall: 18.84%
k=3 | th=3 | head: 26.49% | tail: 39.87% | overall: 34.53%
k=3 | th=4 | head: 26.74% | tail: 40.81% | overall: 35.20%
k=3 | th=5 | head: 26.57% | tail: 40.92% | overall:

In [ ]:
# Define the mapping of k values to their respective threshold lists
k_threshold_mapping = {
    1: [1,2, 3,4, 5,6,7,8,9,10, 20, 30, 40, 50],
    3: [3,4,5,6,7,8,9, 10, 20, 30, 40, 50],
    5: [5,6,7,8,9, 10, 20, 30, 40, 50],
    10: [10,11,12,13,14,15,16,17,18,19, 20, 30, 40, 50]
}

# Iterate through the mapping and print results on the fly
for k, threshold_values in k_threshold_mapping.items():
    for threshold in threshold_values:
        # Evaluate for head
        total_count_head, hit_count_head = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="head")
        head_percentage = hit_count_head / total_count_head * 100

        # Evaluate for tail
        total_count_tail, hit_count_tail = tripleEvaluator.rerank(k=k, model=ensemble_model, threshold=threshold, evaluate_for="tail")
        tail_percentage = hit_count_tail / total_count_tail * 100

        # Calculate overall metrics
        overall_hit_count = hit_count_head + hit_count_tail
        overall_total_count = total_count_head + total_count_tail
        overall_percentage = (overall_hit_count / overall_total_count) * 100

        # Print results immediately
        print(
            f"k={k} | th={threshold} | "
            f"head: {head_percentage:.2f}% | tail: {tail_percentage:.2f}% | overall: {overall_percentage:.2f}%"
        )

k=1 | th=1 | head: 13.82% | tail: 24.38% | overall: 20.16%
k=1 | th=2 | head: 18.49% | tail: 30.98% | overall: 26.00%
k=1 | th=3 | head: 18.20% | tail: 31.56% | overall: 26.23%
k=1 | th=4 | head: 17.85% | tail: 31.33% | overall: 25.95%
k=1 | th=5 | head: 17.07% | tail: 31.02% | overall: 25.45%
k=1 | th=6 | head: 16.73% | tail: 30.94% | overall: 25.27%
k=1 | th=7 | head: 16.10% | tail: 30.56% | overall: 24.79%
k=1 | th=8 | head: 15.57% | tail: 30.56% | overall: 24.58%
k=1 | th=9 | head: 15.23% | tail: 30.10% | overall: 24.16%
k=1 | th=10 | head: 15.09% | tail: 29.64% | overall: 23.84%
k=1 | th=20 | head: 13.20% | tail: 26.97% | overall: 21.48%
k=1 | th=30 | head: 12.12% | tail: 26.32% | overall: 20.65%
k=1 | th=40 | head: 11.60% | tail: 25.82% | overall: 20.14%
k=1 | th=50 | head: 11.37% | tail: 25.20% | overall: 19.68%
k=3 | th=3 | head: 26.49% | tail: 39.87% | overall: 34.53%
k=3 | th=4 | head: 28.92% | tail: 42.72% | overall: 37.21%
k=3 | th=5 | head: 29.20% | tail: 42.96% | overall: